In [ ]:
import os
from typing import Tuple
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import Geod

In [ ]:
# Tratando base de consultores

BASE_DIR = os.getcwd()
BASE_DIR = os.path.dirname(BASE_DIR)  # sobe 1 nível
csv_path = os.path.join(BASE_DIR, "dados", "coordenadas_consultore.csv")
df_consultor = pd.read_csv(csv_path, encoding="latin1", delimiter=",", low_memory=False)

df_consultor['consultor_id'] = df_consultor.index
df_consultor = df_consultor.rename(columns={
    'Latitude': 'latitude',
    'Longitude': 'longitude'
})
print(df_consultor.shape)
df_consultor.head()


In [ ]:
BASE_DIR = os.getcwd()
BASE_DIR = os.path.dirname(BASE_DIR)  # sobe 1 nível
# BASE_DIR = os.path.dirname(os.path.dirname(BASE_DIR))  # sobe 2 níveis
# monta caminho relativo para o CSV
csv_path = os.path.join(BASE_DIR, "dados", "escolas_filtradas.csv")
df_escolas = pd.read_csv(csv_path, encoding="latin1", delimiter=";", low_memory=False)
print(df_escolas.shape)

df_escolas = df_escolas.rename(columns={
    'Latitude': 'latitude',
    'Longitude': 'longitude'
})

df_escolas["AB"] = df_escolas["REC_TOTAL"]

In [ ]:
# Distância geodesica (mais precisa que haversine)
def matriz_calculo_dist_geodesic(lat_e: np.ndarray, lon_e: np.ndarray,
                             lat_c: np.ndarray, lon_c: np.ndarray) -> np.ndarray:
    
    GEOD = Geod(ellps="WGS84")

    lat_e = np.asarray(lat_e, dtype=float)
    lon_e = np.asarray(lon_e, dtype=float)
    lat_c = np.asarray(lat_c, dtype=float)
    lon_c = np.asarray(lon_c, dtype=float)

    E = lat_e.shape[0]
    C = lat_c.shape[0]
    D = np.empty((E, C), dtype=float)

    # Calculo de dist de cada escola para cada consultor
    for j in range(C):
        az1, az2, dist_m = GEOD.inv(lon_e, lat_e, np.full_like(lon_e, lon_c[j]), np.full_like(lat_e, lat_c[j]))
        D[:, j] = dist_m / 1000.0
    return D

In [ ]:
def atribuicao_mais_proximo(df_escolas: pd.DataFrame, dist_mat: np.ndarray) -> pd.DataFrame:
    nearest_idx = np.argmin(dist_mat, axis=1)

    df_assigned = df_escolas.copy()
    df_assigned['cluster'] = nearest_idx
    
    return df_assigned

In [ ]:
# Calculando a matriz de distâncias
dist_mat = matriz_calculo_dist_geodesic(
    df_escolas['latitude'], df_escolas['longitude'],
    df_consultor['latitude'], df_consultor['longitude']
)

# Fazendo a atribuição inicial (Voronoi)
df_escolas_com_cluster = atrib_inicial(df_escolas, dist_mat)

save_path = os.path.join(BASE_DIR, "dados", "escolas_atribuidas_v0.csv")
df_escolas_com_cluster.to_csv(save_path, sep=";", index=False, encoding="utf-8-sig")
